<a href="https://colab.research.google.com/github/vapurv/Cab-fare-prediction/blob/master/cab_fare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [196]:
#By Apurv verma
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split 
import os 
import tensorflow as tf
import tensorboardcolab as tbd
import datetime
import filecmp
import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns 
from collections import Counter 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from pprint import pprint
from sklearn.model_selection import GridSearchCV    


%matplotlib inline

In [197]:
train  = pd.read_csv("train_cab.csv",na_values={"pickup_datetime":"43"})

In [198]:
train.head(1000)

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1.0
1,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1.0
2,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2.0
3,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1.0
4,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1.0
...,...,...,...,...,...,...,...
995,16.9,2011-07-30 07:33:13 UTC,-73.884801,40.755707,-73.980472,40.765556,1.0
996,13.7,2011-09-24 23:20:00 UTC,-73.953603,40.779203,-73.995763,40.726701,3.0
997,29.3,2011-12-22 11:07:00 UTC,-73.942380,40.837712,-73.864372,40.769985,2.0
998,6.5,2014-11-03 12:40:00 UTC,-73.961151,40.774578,-73.972251,40.785640,1.0


In [199]:
test_data = pd.read_csv('test.csv')

In [200]:
test_data


,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1
...,...,...,...,...,...,...
9909,2015-05-10 12:37:51 UTC,-73.968124,40.796997,-73.955643,40.780388,6
9910,2015-01-12 17:05:51 UTC,-73.945511,40.803600,-73.960213,40.776371,6
9911,2015-04-19 20:44:15 UTC,-73.991600,40.726608,-73.789742,40.647011,6
9912,2015-01-31 01:05:19 UTC,-73.985573,40.735432,-73.939178,40.801731,6


*Head Returns the first n rows.*

In [233]:
train.head(5) 

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,Month,Date,Day,Hour
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1.0,2009.0,6.0,15.0,0.0,17.0
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1.0,2010.0,1.0,5.0,1.0,16.0
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2.0,2011.0,8.0,18.0,3.0,0.0
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1.0,2012.0,4.0,21.0,5.0,4.0
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1.0,2010.0,3.0,9.0,1.0,7.0


This will return first five rows of the test data set.

In [234]:
test_data.head(5) 

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,date,month,day,hour
0,-73.973320,40.763805,-73.981430,40.743835,1,2015,2015-01-27,1,1,13
1,-73.986862,40.719383,-73.998886,40.739201,1,2015,2015-01-27,1,1,13
2,-73.982524,40.751260,-73.979654,40.746139,1,2011,2011-10-08,10,5,11
3,-73.981160,40.767807,-73.990448,40.751635,1,2012,2012-12-01,12,5,21
4,-73.966046,40.789775,-73.988565,40.744427,1,2012,2012-12-01,12,5,21


For the better understanding of data,we will check which type of data are present in the training and test data set.


Checking for the type of data in the training set.


In [203]:
train.dtypes

fare_amount           object
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count      float64
dtype: object




**This is formatted as code.**



Checking the type of data in test set.


In [204]:
test_data.dtypes

pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

*Checking the number of rows and column in the training data set.*

In [205]:

print('\t','Number of rows','','Number of colummns')
x,y = train.shape
print('\t',x,'      \t',y)

	 Number of rows  Number of colummns
	 16067       	 7


*Checking the number of rows and column in the test data set.*

In [206]:

print('\t','Number of rows','','Number of colummns')
x,y = test_data.shape
print('\t',x,'      \t',y)

	 Number of rows  Number of colummns
	 9914       	 6


*Describing the data set:Describe(): The describe() method is used for calculating some statistical data like percentile, mean and std of the numerical values of the Series or DataFrame. It analyzes both numeric and object series and also the DataFrame column sets of mixed data types.*

In [207]:

test_data.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000
mean,-73.974722,40.751041,-73.973657,40.751743,1.671273
std,0.042774,0.033541,0.039072,0.035435,1.278747
min,-74.252193,40.573143,-74.263242,40.568973,1.000000
25%,-73.992501,40.736125,-73.991247,40.735254,1.000000
50%,-73.982326,40.753051,-73.980015,40.754065,1.000000
75%,-73.968013,40.767113,-73.964059,40.768757,2.000000
max,-72.986532,41.709555,-72.990963,41.696683,6.000000


*#From the description obatined -values are found are percentile at 25%,50% and 75%.Mean std and minimum are also obtained.*

In [208]:
train.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,16067.000000,16067.000000,16067.000000,16067.000000,16012.000000
mean,-72.462787,39.914725,-72.462328,39.897906,2.625070
std,10.578384,6.826587,10.575062,6.187087,60.844122
min,-74.438233,-74.006893,-74.429332,-74.006377,0.000000
25%,-73.992156,40.734927,-73.991182,40.734651,1.000000
50%,-73.981698,40.752603,-73.980172,40.753567,1.000000
75%,-73.966838,40.767381,-73.963643,40.768013,2.000000
max,40.766125,401.083332,40.802437,41.366138,5345.000000


**Data Cleaning and taking care of missing value**

**On check of the types of data available in the data set we found that cab fare and pickup date time are of object data type.So it need to be converted to numeric**

In [209]:
#Convert fare_amount from object to numeric
train["fare_amount"] = pd.to_numeric(train["fare_amount"],errors = "coerce") 
 

**Checking the train data type again(fare amount must be convverted to numeric data type)**

In [210]:
train.dtypes


fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count      float64
dtype: object

In [211]:
#Converting date time to date_time format from object format
train['pickup_datetime'] =  pd.to_datetime(train['pickup_datetime'], format='%Y-%m-%d %H:%M:%S UTC')

In [212]:
train.dtypes

fare_amount                 float64
pickup_datetime      datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
passenger_count             float64
dtype: object

*There are many rows which have values Na,So we will drop those rows*

In [213]:
train.dropna(subset= ["pickup_datetime"])   #dropping NA values in datetime column

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1.0
1,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1.0
2,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2.0
3,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1.0
4,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1.0
...,...,...,...,...,...,...,...
16062,6.5,2014-12-12 07:41:00,-74.008820,40.718757,-73.998865,40.719987,1.0
16063,16.1,2009-07-13 07:58:00,-73.981310,40.781695,-74.014392,40.715527,2.0
16064,8.5,2009-11-11 11:19:07,-73.972507,40.753417,-73.979577,40.765495,1.0
16065,8.1,2010-05-11 23:53:00,-73.957027,40.765945,-73.981983,40.779560,1.0


In [214]:
x,y =train.shape

In [215]:
x,y

(16067, 7)

 **We will separate the Pickup_datetime column into separate field like year, month, day of the week, etc**



In [216]:


train['year'] = train['pickup_datetime'].dt.year
train['Month'] = train['pickup_datetime'].dt.month
train['Date'] = train['pickup_datetime'].dt.day
train['Day'] = train['pickup_datetime'].dt.dayofweek
train['Hour'] = train['pickup_datetime'].dt.hour
train['Minute'] = train['pickup_datetime'].dt.minute

In [217]:
train.dtypes

fare_amount                 float64
pickup_datetime      datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
passenger_count             float64
year                        float64
Month                       float64
Date                        float64
Day                         float64
Hour                        float64
Minute                      float64
dtype: object

In [218]:
test_data['pickup_datetime']= pd.to_datetime(test_data['pickup_datetime'],format= "%Y-%m-%d %H:%M:%S UTC")

In [219]:
test_data.dtypes

pickup_datetime      datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
passenger_count               int64
dtype: object

**We will seperate pickup date_time column into seperate areas like year,date**


In [220]:
test_data['year'] = test_data['pickup_datetime'].dt.year

In [221]:
test_data.dtypes

pickup_datetime      datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
passenger_count               int64
year                          int64
dtype: object

In [222]:
test_data['date'] = test_data['pickup_datetime'].dt.date
test_data['month']=test_data['pickup_datetime'].dt.month
test_data['day'] = test_data['pickup_datetime'].dt.dayofweek
test_data['hour'] = test_data['pickup_datetime'].dt.hour
test_data['Minute'] = test_data['pickup_datetime'].dt.minute
 




In [223]:
test_data.dtypes

pickup_datetime      datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
passenger_count               int64
year                          int64
date                         object
month                         int64
day                           int64
hour                          int64
Minute                        int64
dtype: object

**Now we have splitted the pickup date time variable into different varaibles like month, year, day etc so now we dont need to have that pickup_Date variable now.**

**

In [230]:
#drop = ['pickup_datetime',  'Minute']
#train = train.drop(drop, axis = 1)



In [231]:
#drop1=['pickup_datetime',  'Minute']
#test_data = test_data.drop(drop, axis = 1)

**Check for the un-necessary data in the train data set**

1>Passenger count should be not more than 6.

2>Longitude should be in the range of -90 to +90.

3>Latitude should be in the range of -180 to +180.

4>Some values which does not seem acceptable,we will remove them.


In [226]:
train['passenger_count'].describe()

count    16012.000000
mean         2.625070
std         60.844122
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max       5345.000000
Name: passenger_count, dtype: float64

**In the above description of the passenger count we see that max number of passenger count is 5435 which is absurd.So we will remove all those row who have value more than 6.**

In [227]:
train = train.drop(train[train["passenger_count"]> 6 ].index, axis=0)

In [228]:
train['passenger_count'].describe()

count    15992.000000
mean         1.643661
std          1.267474
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max          6.000000
Name: passenger_count, dtype: float64

**removing datetime missing values rows.**

In [232]:
train = train.drop(train[train['pickup_datetime'].isnull()].index, axis=0)
print(train.shape)
print(train['pickup_datetime'].isnull().sum())

KeyError: ignored

**Also removing the values which have the passenger count equal to 0.**

In [ ]:
train = train.drop(train[train["passenger_count"] == 0 ].index, axis=0)

In [ ]:
train["passenger_count"].describe()

In [ ]:
train['passenger_count'].sort_values(ascending = True)

**We observe in sorted passenger count that many values are zero so we will remove them.**

In [ ]:
train = train.drop(train[train['passenger_count'].isnull()].index, axis=0)
print(train.shape)
print(train['passenger_count'].isnull().sum())

One have passenger count 0.12,which is not possible,so we will remove them.

In [ ]:

train = train.drop(train[train["passenger_count"] == 0.12 ].index, axis=0)
print(train.shape)
print(train['passenger_count'].isnull().sum())


**Checking for the longitude and latitude.**




**We will drop all latitude who are not in the range of -90 to +90.**

**We will also drop all longitude who are not in the range of +180 to -180.**


In [ ]:
train[train['pickup_latitude']<-90]
train[train['pickup_latitude']>90]

In [ ]:
#dropping one value of >90
train = train.drop((train[train['pickup_latitude']<-90]).index, axis=0)
train = train.drop((train[train['pickup_latitude']>90]).index, axis=0)

In [ ]:
train[train['pickup_longitude']<-180]
train[train['pickup_longitude']>180]

In [ ]:
train[train['dropoff_latitude']<-90]
train[train['dropoff_latitude']>90]


In [ ]:
train[train['dropoff_longitude']<-180]
train[train['dropoff_longitude']>180]


In [ ]:
train['dropoff_longitude'].describe()

In [ ]:
train['dropoff_latitude'].describe()

In [ ]:
train['pickup_latitude'].describe()

In [ ]:
train['dropoff_longitude'].describe()

**Checking for the fare.**

In [ ]:
train['fare_amount'].sort_values(ascending=True)

**A lot of fare amount are in negative and many are also null,so we will remove them.**

In [ ]:
train = train.drop(train[train['fare_amount'].isnull()].index,axis=0)

In [ ]:

train = train.drop(train[train["fare_amount"]<0].index, axis=0)


In [ ]:
train = train.drop(train[train["fare_amount"]<1].index, axis=0)
train = train.drop(train[train["fare_amount"]> 454 ].index, axis=0)

In [ ]:
train['fare_amount'].describe()

In [ ]:
x,y = train.shape
print(x,y)

**Calculating distance between two points based on the given co-ordinates**

In [ ]:
from math import radians, cos, sin, asin, sqrt
def haversine(a):
    longitude1=a[0]
    latitude1=a[1]
    longitude2=a[2]
    latitude2=a[3]
    
    longitude1, latitude1, longitude2, latitude2 = map(radians, [longitude1, latitude1, longitude2, latitude2])
    
    # haversine formula 
    dlong = longitude2 - longitude1 
    dlati = latitude2 - latitude1 
    a = sin(dlati/2)**2 + cos(latitude1) * cos(latitude2) * sin(dlong/2)**2
    c =  2 * asin(sqrt(a))
    # Earth radius = 6,371
    km = 6371* c
    return km

In [ ]:
train['distance'] = train[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']].apply(haversine,axis=1)

In [ ]:
test_data['distance'] = test_data[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']].apply(haversine,axis=1)

**we have created distance using pickup and drop longitudes and latitudes so we will also drop pickup and drop longitudes and latitudes variables.**

In [ ]:
drop = [ 'pickup_longitude', 'pickup_latitude','dropoff_longitude', 'dropoff_latitude']
train = train.drop(drop, axis = 1)

In [ ]:
drop1=['pickup_longitude', 'pickup_latitude','dropoff_longitude', 'dropoff_latitude']
test_data = test_data.drop(drop, axis = 1)

**Checking the nature of the updated data**

In [ ]:
train.head()

In [ ]:
test_data.head(5)

In [ ]:
test_data.nunique()

In [ ]:
train.nunique()

In [ ]:
train['distance'].sort_values( ascending=False,kind = 'quicksort')

**In the sorted distance table we see that there are a lot of distance which are zero and those data are not of any use for the prediction.**

**Some values at the top of the table shows that distance travelled are more than 8k km and coming down to the table we get to see that the distance become ariound 127 km, so there is huge diff in the data available, so we will remove them.**

*Removing the zero distance.*

In [ ]:
train = train.drop(train[train['distance']==0].index,level = None,axis = 0)

*Removing the distance which are more than 129 kms*


In [ ]:
train = train.drop(train[train['distance']>129].index,axis = 0,level = None)

In [ ]:
train['distance']

In [ ]:
train.describe()

In [ ]:
train.dtypes

**From the above data description we found that passenger count,year,month,date,day are in float datatype which is irrelevent.So we will convert all those in int data type.**

In [ ]:
train['passenger_count']

In [ ]:
train['passenger_count'] = train['passenger_count'].astype('int64')


In [ ]:
train['year'].describe()

In [ ]:
train = train.drop(train[train['year'].isnull()].index, axis=0)


In [ ]:
train['Month'] = train['Month'].astype('int64')
train['Date'] = train['Date'].astype('int64')
train['Day'] = train['Day'].astype('int64')
train['Hour'] = train['Hour'].astype('int64')

In [ ]:
train['year'] = train['year'].astype('int64')

In [ ]:
train.dtypes

In [ ]:
train['year']

In [ ]:
train.head(5)

In [ ]:
test_data.dtypes

In [ ]:
test_data.head()

**We are done with data cleaning part**

**Applying Machine Learning Algorithms**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( train.iloc[:, train.columns != 'fare_amount'], 
                         train.iloc[:, 0], test_size = 0.20, random_state = 1)

In [ ]:
X_train

In [ ]:
y_test

In [ ]:
X_train.shape

In [ ]:
X_train.describe()

In [ ]:
X_test.describe()

In [ ]:
X_test.shape

In [ ]:
y_train

**Applying linear model**

In [ ]:
regressor = LinearRegression().fit(X_train,y_train)

In [ ]:
y_pred= regressor.predict(X_test)
print(y_pred)

In [ ]:
test_data['predicted fare ']= pd.Series(y_pred)

In [ ]:
test_data.head(1000)

***Implementing the Decision tree model.***

In [ ]:
Decision_tree_regressor = DecisionTreeRegressor().fit(X_train,y_train)

In [ ]:
y_pred= Decision_tree_regressor.predict(X_test)

In [ ]:
test_data['predicted fare ']= pd.Series(y_pred)

In [ ]:
test_data.head(1000)